In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from tensorflow.python.keras.backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tcn import TCN, tcn_full_summary
from tcn import compiled_tcn
from tensorflow.keras.utils import to_categorical
import csv
import random
import itertools
from keras_flops import get_flops
from mango.tuner import Tuner
import time
import pickle
from hardware_utils import *
from data_utils import *

# Import Dataset

In [ ]:
sampling_rate = 100
window_size = 550
stride = 50
f = '/home/nesl/earable/Earable/Activity_Dataset/' #dataset directory

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=0)

print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

# Training and NAS

In [ ]:
dirpath="/home/nesl/Mbed Programs/auritus_tcn/" #hardware program directory - this is where the TCN deployment code is stored
device = "NUCLEO_F446RE" #which hardware to use
model_name = 'Auritus_HIL'+device+'.hdf5'
HIL = True #use real hardware or proxy?
quantization = False #use quantization or not?
model_epochs = 900 #epochs to train each model for
NAS_epochs = 50 #epochs for hyperparameter tuning
output_name = 'g_model.tflite'
log_file_name = 'TCN_Auritus_'+device+'.csv'
if os.path.exists(log_file_name):
    os.remove(log_file_name)
row_write = ['score', '1-accuracy','RAM','Flash','FLOPS','Latency','nb_filters','kernel_size',
             'dilations','use_skip_connections']
with open(log_file_name, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_write)
if os.path.exists(log_file_name[0:-4]+'.p'):
    os.remove(log_file_name[0:-4]+'.p')

In [ ]:
def objective_NN(epochs=1000,nb_filters=10,kernel_size=3,
                 dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256],
                 use_skip_connections=True):
    
    err = 'inf'
    input_dim=X_tr.shape[2]
    
    model = compiled_tcn(return_sequences=False,
                         num_feat=input_dim,
                         num_classes=Y_tr.shape[1],
                         nb_filters=nb_filters,
                         kernel_size=kernel_size,
                         dilations=dilations,
                         nb_stacks=1,
                         max_len=window_size,
                         use_weight_norm=False,
                         use_skip_connections=use_skip_connections)
    
    opt = tf.keras.optimizers.Adam()
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=opt,metrics=['accuracy'])
    Flops = get_flops(model, batch_size=1)
    convert_to_tflite_model(model=model,training_data=X_tr,quantization=quantization,output_name=output_name) 
    maxRAM, maxFlash = return_hardware_specs(device)
    
    if(HIL==True):
        convert_to_cpp_model(dirpath)
        RAM, Flash, Latency, idealArenaSize, errorCode = HIL_controller(dirpath=dirpath,
                                                                       chosen_device=device,
                                                                       window_size=window_size, 
                                                                    number_of_channels = input_dim,
                                                                   quantization=quantization)     
        score = -5.0
        if(Flash==-1):
            row_write = [score, err,RAM,Flash,Flops,Latency,
                 nb_filters,kernel_size,dilations,use_skip_connections]
            print('Design choice:',row_write)
            with open(log_file_name, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(row_write)
            return score    
    
        elif(Flash!=-1):
            checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', verbose=1, save_best_only=True)
            model.fit(x=X_tr, y=Y_tr,validation_split=0.1,
                      epochs=epochs,callbacks=[checkpoint],shuffle=True,verbose=1)
            err = 1-checkpoint.best
            resource_usage = (RAM/maxRAM) + (Flash/maxFlash) 
            score = -err + 0.01*resource_usage - 0.05*Latency #weigh each component as you like
                
            row_write = [score, err,RAM,Flash,Flops,Latency,
                 nb_filters,kernel_size,dilations,use_skip_connections]
            print('Design choice:',row_write)
            with open(log_file_name, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(row_write)   
    else:
        score = -5.0
        Flash = os.path.getsize(output_name)
        RAM = get_model_memory_usage(batch_size=1,model=model)
        Latency=-1
        max_flops = (30e6)
        
        if(RAM < maxRAM and Flash<maxFlash):
            checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', verbose=1, save_best_only=True)
            model.fit(x=X_tr, y=Y_tr,validation_split=0.1,
                      epochs=epochs,callbacks=[checkpoint],shuffle=True,verbose=1)
            err = 1-checkpoint.best 
            resource_usage = (RAM/maxRAM) + (Flash/maxFlash)
            score = -err + 0.01*resource_usage - 0.05*(Flops/max_flops) #weigh each component as you like
        
        row_write = [score, err,RAM,Flash,Flops,Latency,
                 nb_filters,kernel_size,dilations,use_skip_connections]
        print('Design choice:',row_write)
        with open(log_file_name, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row_write) 
    
    return score        

In [ ]:
import pickle 

def save_res(data, file_name):
    pickle.dump( data, open( file_name, "wb" ) )
    
min_layer = 3
max_layer = 8
a_list = [1,2,4,8,16,32,64,128,256]
all_combinations = []
dil_list = []
for r in range(len(a_list) + 1):
    combinations_object = itertools.combinations(a_list, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list
all_combinations = all_combinations[1:]
for item in all_combinations:
    if(len(item) >= min_layer and len(item) <= max_layer):
        dil_list.append(list(item))

param_dict = {
    'nb_filters': range(2,64),
    'kernel_size': range(2,16),
    'use_skip_connections': [True, False],
    'dil_list': dil_list
}

def objfunc(args_list):

    objective_evaluated = []
    
    start_time = time.time()
    
    for hyper_par in args_list:
        nb_filters = hyper_par['nb_filters']
        kernel_size = hyper_par['kernel_size']
        use_skip_connections = hyper_par['use_skip_connections']
        dil_list = hyper_par['dil_list']
            
        objective = objective_NN(epochs=model_epochs,nb_filters=nb_filters,kernel_size=kernel_size,
                                 dilations=dil_list,use_skip_connections=use_skip_connections,)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        
    return objective_evaluated

conf_Dict = dict()
conf_Dict['batch_size'] = 1 
conf_Dict['num_iteration'] = NAS_epochs
conf_Dict['initial_random']= 5
tuner = Tuner(param_dict, objfunc,conf_Dict)
all_runs = []
results = tuner.maximize()
all_runs.append(results)
save_res(all_runs,log_file_name[0:-4]+'.p')

# Train the best model

In [ ]:
input_dim=X_tr.shape[2]

model = compiled_tcn(return_sequences=False,
                     num_feat=input_dim,
                     num_classes=Y_tr.shape[1],
                     nb_filters=results['best_params']['nb_filters'],
                     kernel_size=results['best_params']['kernel_size'],
                     dilations=results['best_params']['dilations'],
                     nb_stacks=1,
                     max_len=window_size,
                     use_weight_norm=False,
                     use_skip_connections=results['best_params']['use_skip_connections'])

opt = tf.keras.optimizers.Adam()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=opt,metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', verbose=1, save_best_only=True)
model.fit(x=X_tr, y=Y_tr,validation_split=0.1,
          epochs=model_epochs,callbacks=[checkpoint],shuffle=True,verbose=1)

model = load_model(model_name,custom_objects={'TCN': TCN})
test_accu = model.evaluate(x=X_test,y=Y_test)[1]
print('Test Accuracy:', test_accu)

# Deployment

### Conversion to TFLite

In [ ]:
convert_to_tflite_model(model=model,training_data=X_tr,quantization=quantization,output_name=output_name) 

### Conversion to C++

In [ ]:
convert_to_cpp_model(dirpath)